In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class VideoFrameDataset(Dataset):
    def __init__(self, data_dir, sequence_length=5, frame_size=(64, 64)):
        self.data_dir = Path(data_dir)
        self.sequence_length = sequence_length
        self.frame_size = frame_size
        
        self.video_files = list(self.data_dir.glob("**/*.mp4"))
        print(f"Found {len(self.video_files)} video files")
        
        self.sequences = []
        self._build_sequences()
        
    def _build_sequences(self):
        for video_path in tqdm(self.video_files[:100], desc="Processing videos"):
            try:
                cap = cv2.VideoCapture(str(video_path))
                frames = []
                
                while True:
                    ret, frame = cap.read()
                    if not ret:
                        break
                        
                    frame = cv2.resize(frame, self.frame_size)
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    frame = frame.astype(np.float32) / 255.0
                    frames.append(frame)
                
                cap.release()
                
                for i in range(len(frames) - self.sequence_length):
                    sequence = frames[i:i + self.sequence_length]
                    self.sequences.append(np.array(sequence))
                    
            except Exception as e:
                print(f"Error processing {video_path}: {e}")
                continue
        
        print(f"Created {len(self.sequences)} frame sequences")
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        input_frames = torch.FloatTensor(sequence[:-1]).permute(0, 3, 1, 2)
        target_frame = torch.FloatTensor(sequence[-1]).permute(2, 0, 1)
        return input_frames, target_frame

class MotionEstimator(nn.Module):
    """Estimates optical flow between frames"""
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(6, 32, 3, padding=1),  # 6 = 3*2 frames
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 2, 3, padding=1)  # 2 channels for x,y flow
        )
        
    def forward(self, frame1, frame2):
        x = torch.cat([frame1, frame2], dim=1)
        flow = self.encoder(x)
        return flow

class ErrorPredictor(nn.Module):
    """Predicts where errors are likely to occur"""
    def __init__(self):
        super().__init__()
        self.detector = nn.Sequential(
            nn.Conv2d(9, 32, 3, padding=1),  # 3 frames * 3 channels
            nn.ReLU(),
            nn.Conv2d(32, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 1, 3, padding=1),
            nn.Sigmoid()
        )
        
    def forward(self, prev_frame, predicted_frame, expected_frame):
        x = torch.cat([prev_frame, predicted_frame, expected_frame], dim=1)
        error_map = self.detector(x)
        return error_map

class SelfCorrectingPredictor(nn.Module):
    """Main predictor with built-in error correction"""
    def __init__(self, sequence_length=4):
        super().__init__()
        
        # FIXED: Basic frame predictor with correct dimensions
        self.predictor = nn.Sequential(
            nn.Conv3d(3, 32, (3, 3, 3), padding=1),
            nn.ReLU(),
            nn.Conv3d(32, 64, (3, 3, 3), padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool3d((1, None, None)),  # This reduces temporal dimension to 1
            nn.Conv3d(64, 32, (1, 3, 3), padding=(0, 1, 1)),  # 1x3x3 conv for spatial only
            nn.ReLU(),
            nn.Conv3d(32, 3, (1, 3, 3), padding=(0, 1, 1)),   # Final layer
            nn.Sigmoid()
        )
        
        # Motion estimator
        self.motion_estimator = MotionEstimator()
        
        # Error predictor
        self.error_predictor = ErrorPredictor()
        
        # Corrector network - FIXED: 6 input channels (3 predicted + 1 error_map + 2 flow)
        self.corrector = nn.Sequential(
            nn.Conv2d(6, 32, 3, padding=1),  # predicted + error_map + motion (3+1+2=6)
            nn.ReLU(),
            nn.Conv2d(32, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 3, 3, padding=1),
            nn.Tanh()  # Correction residuals
        )
        
    def warp_frame(self, frame, flow):
        """Warp frame using optical flow"""
        B, C, H, W = frame.shape
        
        # Create coordinate grid
        y, x = torch.meshgrid(torch.arange(H), torch.arange(W), indexing='ij')
        grid = torch.stack([x, y], dim=0).float().to(frame.device)
        grid = grid.unsqueeze(0).repeat(B, 1, 1, 1)
        
        # Apply flow
        new_grid = grid + flow
        
        # Normalize for grid_sample
        new_grid[:, 0] = 2.0 * new_grid[:, 0] / (W - 1) - 1.0
        new_grid[:, 1] = 2.0 * new_grid[:, 1] / (H - 1) - 1.0
        new_grid = new_grid.permute(0, 2, 3, 1)
        
        # Warp
        warped = F.grid_sample(frame, new_grid, align_corners=True, padding_mode='border')
        return warped
        
    def forward(self, input_sequence):
        # FIXED: Handle dimensions correctly
        x = input_sequence.permute(0, 2, 1, 3, 4)  # (B, C, T, H, W)
        
        # Get initial prediction
        pred_5d = self.predictor(x)  # Output shape: (B, 3, 1, H, W)
        initial_pred = pred_5d.squeeze(2)  # Remove temporal dimension: (B, 3, H, W)
        
        # Estimate motion from last two frames
        last_frame = input_sequence[:, -1]      # (B, 3, H, W)
        second_last = input_sequence[:, -2]     # (B, 3, H, W)
        flow = self.motion_estimator(second_last, last_frame)  # (B, 2, H, W)
        
        # Create expected frame by warping
        expected_frame = self.warp_frame(last_frame, flow)
        
        # Predict where errors might occur
        error_map = self.error_predictor(last_frame, initial_pred, expected_frame)  # (B, 1, H, W)
        
        # Generate corrections - FIXED: Concatenate with correct dimensions
        correction_input = torch.cat([
            initial_pred,  # (B, 3, H, W)
            error_map,     # (B, 1, H, W) 
            flow           # (B, 2, H, W)
        ], dim=1)  # Total: (B, 6, H, W)
        
        # Adjust corrector to accept 6 channels instead of 7
        corrections = self.corrector(correction_input)
        
        # Apply corrections weighted by error map
        corrected_frame = initial_pred + corrections * error_map
        corrected_frame = torch.clamp(corrected_frame, 0, 1)
        
        return {
            'prediction': corrected_frame,
            'initial_prediction': initial_pred,
            'flow': flow,
            'error_map': error_map,
            'expected_frame': expected_frame,
            'corrections': corrections
        }

class SelfCorrectingLoss(nn.Module):
    """Loss that encourages good corrections"""
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, outputs, target):
        prediction = outputs['prediction']
        initial_pred = outputs['initial_prediction']
        error_map = outputs['error_map']
        expected_frame = outputs['expected_frame']
        
        # Main reconstruction loss
        recon_loss = self.mse(prediction, target)
        
        # Error map should predict actual errors
        actual_error = torch.abs(initial_pred - target).mean(dim=1, keepdim=True)
        error_loss = self.mse(error_map, actual_error)
        
        # Motion consistency loss
        motion_loss = self.mse(expected_frame, target)
        
        # Correction should improve upon initial prediction
        initial_loss = self.mse(initial_pred, target)
        improvement = torch.clamp(initial_loss - recon_loss, min=0)
        correction_loss = -improvement  # Reward improvements
        
        total_loss = (recon_loss + 
                     0.1 * error_loss + 
                     0.1 * motion_loss + 
                     0.05 * correction_loss)
        
        return total_loss

class VideoContinuator:
    """Simple video continuator with self-correction"""
    def __init__(self, model_path=None, device='cuda', frame_size=(64, 64), sequence_length=4):
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        self.frame_size = frame_size
        self.sequence_length = sequence_length
        
        # Load model
        self.model = SelfCorrectingPredictor(sequence_length).to(self.device)
        if model_path and os.path.exists(model_path):
            try:
                self.model.load_state_dict(torch.load(model_path, map_location=self.device))
                print(f"Loaded model from {model_path}")
            except Exception as e:
                print(f"Error loading model: {e}, using untrained model")
        else:
            print("Using untrained model")
            
        self.model.eval()
        
        # Drift detection
        self.error_history = []
        self.correction_threshold = 0.15
        
    def load_video_frames(self, video_path, max_frames=None):
        cap = cv2.VideoCapture(video_path)
        frames = []
        
        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            frame = cv2.resize(frame, self.frame_size)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = frame.astype(np.float32) / 255.0
            frames.append(frame)
            
            frame_count += 1
            if max_frames and frame_count >= max_frames:
                break
                
        cap.release()
        return frames
        
    def predict_next_frame(self, input_frames):
        # FIXED: Convert list to numpy array first, then to tensor
        input_array = np.array(input_frames)  # Convert list to numpy array
        input_tensor = torch.FloatTensor(input_array).unsqueeze(0)  # Then to tensor
        input_tensor = input_tensor.permute(0, 1, 4, 2, 3).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(input_tensor)
            
        prediction = outputs['prediction'].squeeze(0).cpu().permute(1, 2, 0).numpy()
        error_map = outputs['error_map'].squeeze(0).cpu().numpy()
        
        # Track error levels
        avg_error = np.mean(error_map)
        self.error_history.append(avg_error)
        if len(self.error_history) > 10:
            self.error_history.pop(0)
            
        return np.clip(prediction, 0, 1), avg_error
        
    def detect_drift(self):
        """Simple drift detection based on error trends"""
        if len(self.error_history) < 5:
            return False
            
        recent_error = np.mean(self.error_history[-3:])
        return recent_error > self.correction_threshold
        
    def continue_video(self, video_path, num_frames=30, output_path=None):
        if output_path is None:
            output_path = video_path.replace('.mp4', '_self_corrected.mp4')
            
        frames = self.load_video_frames(video_path)
        if len(frames) < self.sequence_length:
            raise ValueError(f"Need at least {self.sequence_length} frames")
            
        current_sequence = frames[-self.sequence_length:]
        all_frames = frames.copy()
        
        print(f"Generating {num_frames} frames with self-correction...")
        
        corrections_applied = 0
        for i in tqdm(range(num_frames)):
            predicted_frame, error_level = self.predict_next_frame(current_sequence)
            
            # Check for drift
            if self.detect_drift() and i > 5:
                print(f"Frame {i}: High error detected ({error_level:.3f}), applying correction")
                
                # Simple correction: blend with previous frame
                correction_strength = min(0.3, error_level)
                predicted_frame = (predicted_frame * (1 - correction_strength) + 
                                 current_sequence[-1] * correction_strength)
                corrections_applied += 1
                
            all_frames.append(predicted_frame)
            current_sequence = current_sequence[1:] + [predicted_frame]
            
        print(f"Applied {corrections_applied} corrections out of {num_frames} frames")
        
        self.save_video(all_frames, output_path)
        return all_frames
        

    def save_video(self, frames, output_path, fps=24):
        """Save frames as video with better error handling and debugging"""
        try:
            print(f"Attempting to save {len(frames)} frames to {output_path}")
            
            # Ensure output directory exists
            output_dir = os.path.dirname(output_path)
            os.makedirs(output_dir, exist_ok=True)
            
            if not frames:
                print("ERROR: No frames to save!")
                return False
            
            # Debug first frame
            first_frame = frames[0]
            print(f"First frame - shape: {first_frame.shape}, dtype: {first_frame.dtype}")
            print(f"First frame - min: {first_frame.min():.3f}, max: {first_frame.max():.3f}")
            
            # Get frame dimensions
            height, width = first_frame.shape[:2]
            print(f"Video dimensions: {width}x{height}")
            
            # Process all frames first to ensure they're in correct format
            processed_frames = []
            for i, frame in enumerate(frames):
                try:
                    # Ensure frame is numpy array
                    if not isinstance(frame, np.ndarray):
                        frame = np.array(frame)
                    
                    # Handle different input ranges
                    if frame.max() <= 1.0:
                        # Frame is in 0-1 range, convert to 0-255
                        frame_uint8 = (np.clip(frame, 0, 1) * 255).astype(np.uint8)
                    else:
                        # Frame is already in 0-255 range
                        frame_uint8 = np.clip(frame, 0, 255).astype(np.uint8)
                    
                    # Ensure it's 3-channel RGB
                    if len(frame_uint8.shape) == 2:
                        # Grayscale to RGB
                        frame_uint8 = cv2.cvtColor(frame_uint8, cv2.COLOR_GRAY2RGB)
                    elif frame_uint8.shape[2] == 4:
                        # RGBA to RGB
                        frame_uint8 = frame_uint8[:, :, :3]
                    
                    # Convert RGB to BGR for OpenCV
                    frame_bgr = cv2.cvtColor(frame_uint8, cv2.COLOR_RGB2BGR)
                    processed_frames.append(frame_bgr)
                    
                    # Debug first few frames
                    if i < 3:
                        print(f"Frame {i} processed - shape: {frame_bgr.shape}, dtype: {frame_bgr.dtype}, range: [{frame_bgr.min()}, {frame_bgr.max()}]")
                        
                except Exception as frame_error:
                    print(f"Error processing frame {i}: {frame_error}")
                    return False
            
            print(f"Successfully processed {len(processed_frames)} frames")
            
            # Try saving with different methods
            methods = [
                ('mp4v', '.mp4'),
                ('XVID', '.avi'),
                ('MJPG', '.avi')
            ]
            
            for codec_name, extension in methods:
                try:
                    if extension != '.mp4':
                        test_output = output_path.replace('.mp4', extension)
                    else:
                        test_output = output_path
                    
                    print(f"Trying codec: {codec_name}")
                    fourcc = cv2.VideoWriter_fourcc(*codec_name)
                    
                    # Create VideoWriter
                    out = cv2.VideoWriter(test_output, fourcc, fps, (width, height))
                    
                    if not out.isOpened():
                        print(f"Failed to initialize VideoWriter with {codec_name}")
                        continue
                    
                    # Write frames
                    frames_written = 0
                    for i, frame_bgr in enumerate(processed_frames):
                        ret = out.write(frame_bgr)
                        if ret:
                            frames_written += 1
                        else:
                            print(f"Failed to write frame {i}")
                            # Don't break, try to continue
                    
                    out.release()
                    
                    # Check result
                    if os.path.exists(test_output):
                        file_size = os.path.getsize(test_output)
                        print(f"Codec {codec_name}: wrote {frames_written}/{len(processed_frames)} frames, file size: {file_size} bytes")
                        
                        if file_size > 10000:  # At least 10KB for a real video
                            print(f"SUCCESS: Video saved successfully with {codec_name}")
                            return True
                        else:
                            print(f"File too small, trying next codec...")
                            if os.path.exists(test_output):
                                os.remove(test_output)
                    
                except Exception as codec_error:
                    print(f"Error with {codec_name}: {codec_error}")
                    continue
            
            # If all codecs fail, try a simple approach - save frames as images and create video using ffmpeg-like approach
            print("All video codecs failed, trying alternative approach...")
            
            # Alternative: Save as uncompressed AVI
            try:
                alt_output = output_path.replace('.mp4', '_uncompressed.avi')
                fourcc = 0  # Uncompressed
                out = cv2.VideoWriter(alt_output, fourcc, fps, (width, height))
                
                if out.isOpened():
                    for frame_bgr in processed_frames:
                        out.write(frame_bgr)
                    out.release()
                    
                    if os.path.exists(alt_output):
                        file_size = os.path.getsize(alt_output)
                        if file_size > 10000:
                            print(f"SUCCESS: Uncompressed video saved: {file_size} bytes")
                            return True
            except:
                pass
                
            print("ERROR: All video saving methods failed!")
            return False
            
        except Exception as e:
            print(f"ERROR in save_video: {e}")
            import traceback
            traceback.print_exc()
            return False        

def train_model(train_loader, num_epochs=10):
    model = SelfCorrectingPredictor().to(device)
    
    # FIXED: Update corrector to accept 6 channels instead of 7
    model.corrector[0] = nn.Conv2d(6, 32, 3, padding=1).to(device)  # Fix input channels
    
    criterion = SelfCorrectingLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch_idx, (input_frames, target_frame) in enumerate(tqdm(train_loader)):
            input_frames = input_frames.to(device)
            target_frame = target_frame.to(device)
            
            optimizer.zero_grad()
            
            try:
                outputs = model(input_frames)
                loss = criterion(outputs, target_frame)
                loss.backward()
                optimizer.step()
                
                epoch_loss += loss.item()
            except Exception as e:
                print(f"Error in batch {batch_idx}: {e}")
                continue
                
        print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(train_loader):.4f}")
        torch.save(model.state_dict(), f'/kaggle/working/corrected_model_epoch_{epoch+1}.pth')
    
    return model

# Updated main execution section
if __name__ == "__main__":
    print("Self-Correcting Video Continuation System")
    
    # Create output directory
    import os
    os.makedirs('/kaggle/output', exist_ok=True)
    print("Created /kaggle/output directory")
    
    # Video continuation
    video_files = glob.glob('/kaggle/input/my-videos-for-continuation/*.mp4')
    print(f"Found video files: {video_files}")
    
    if video_files:
        print(f"Found {len(video_files)} videos to process")
        continuator = VideoContinuator()
        
        for video_path in video_files[:2]:  # Process first 2 videos
            print(f"\nProcessing {video_path}...")
            try:
                output_filename = os.path.basename(video_path).replace(".mp4", "_corrected.mp4")
                output_path = f'/kaggle/output/{output_filename}'
                print(f"Output path: {output_path}")
                
                # Generate more frames for a substantial video
                continued_frames = continuator.continue_video(
                    video_path=video_path,
                    num_frames=30,  # Generate 30 new frames (about 1 second at 30fps)
                    output_path=output_path
                )
                
                # Check if file was created successfully
                if os.path.exists(output_path):
                    output_size = os.path.getsize(output_path)
                    print(f"SUCCESS: Output video created! Size: {output_size} bytes")
                    if output_size < 10000:
                        print("WARNING: File size seems too small - there may be an issue with frame encoding")
                else:
                    print(f"ERROR: Output video was not created at {output_path}")
                    
            except Exception as e:
                print(f"ERROR processing {video_path}: {e}")
                import traceback
                traceback.print_exc()
    
    # Check final output
    if os.path.exists('/kaggle/output'):
        output_files = os.listdir('/kaggle/output')
        print(f"\nFinal contents of /kaggle/output: {output_files}")
        for file in output_files:
            file_path = f'/kaggle/output/{file}'
            if os.path.isfile(file_path):
                size = os.path.getsize(file_path)
                print(f"  {file}: {size} bytes")
                
                # Try to get video info
                try:
                    cap = cv2.VideoCapture(file_path)
                    if cap.isOpened():
                        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                        fps = cap.get(cv2.CAP_PROP_FPS)
                        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        print(f"    Video info: {frame_count} frames, {fps} fps, {width}x{height}")
                        cap.release()
                    else:
                        print(f"    Could not open video file")
                except Exception as video_error:
                    print(f"    Error reading video info: {video_error}")
    
    print("\nProcess complete!")

# Add this to the end of your script to ensure files are properly accessible

import os
import shutil
import glob

print("=== ENSURING FILES ARE DOWNLOADABLE ===")

# Check what's actually in the directories
directories_to_check = ['/kaggle/output', '/kaggle/working', '.']

for directory in directories_to_check:
    if os.path.exists(directory):
        files = os.listdir(directory)
        print(f"\nContents of {directory}:")
        for file in files:
            file_path = os.path.join(directory, file)
            if os.path.isfile(file_path):
                size = os.path.getsize(file_path)
                print(f"  {file}: {size} bytes")

# Copy any MP4 files to both working and output directories
mp4_files = []
for directory in ['/kaggle/output', '/kaggle/working', '.']:
    if os.path.exists(directory):
        mp4_files.extend(glob.glob(os.path.join(directory, '*.mp4')))

print(f"\nFound MP4 files: {mp4_files}")

# Ensure output directory exists and copy files there
os.makedirs('/kaggle/output', exist_ok=True)
os.makedirs('/kaggle/working', exist_ok=True)

for mp4_file in mp4_files:
    filename = os.path.basename(mp4_file)
    
    # Copy to both directories to be safe
    for target_dir in ['/kaggle/output', '/kaggle/working']:
        target_path = os.path.join(target_dir, filename)
        if not os.path.exists(target_path):
            try:
                shutil.copy2(mp4_file, target_path)
                print(f"Copied {filename} to {target_dir}")
            except Exception as e:
                print(f"Error copying to {target_dir}: {e}")

# Final verification
print("\n=== FINAL VERIFICATION ===")
for directory in ['/kaggle/output', '/kaggle/working']:
    if os.path.exists(directory):
        files = [f for f in os.listdir(directory) if f.endswith('.mp4')]
        print(f"\nMP4 files in {directory}: {files}")
        for file in files:
            file_path = os.path.join(directory, file)
            size = os.path.getsize(file_path)
            print(f"  {file}: {size} bytes")
            
            # Try to verify it's a valid video
            try:
                import cv2
                cap = cv2.VideoCapture(file_path)
                if cap.isOpened():
                    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    fps = cap.get(cv2.CAP_PROP_FPS)
                    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                    print(f"    ✓ Valid video: {frame_count} frames, {fps} fps, {width}x{height}")
                    cap.release()
                else:
                    print(f"    ✗ Cannot open as video")
            except Exception as e:
                print(f"    ✗ Video validation error: {e}")

# Try alternative approach - create a simple test video to verify the system works
print("\n=== CREATING TEST VIDEO ===")
try:
    import cv2
    import numpy as np
    
    # Create a simple test video
    test_output = '/kaggle/output/test_video.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(test_output, fourcc, 10.0, (64, 64))
    
    if out.isOpened():
        # Create 30 test frames with different colors
        for i in range(30):
            # Create a solid color frame that changes over time
            color = int(255 * i / 30)
            frame = np.full((64, 64, 3), [color, 128, 255-color], dtype=np.uint8)
            out.write(frame)
        
        out.release()
        
        if os.path.exists(test_output):
            test_size = os.path.getsize(test_output)
            print(f"✓ Test video created successfully: {test_size} bytes")
            
            # Copy to working directory too
            shutil.copy2(test_output, '/kaggle/working/test_video.mp4')
            print("✓ Test video copied to working directory")
        else:
            print("✗ Test video creation failed")
    else:
        print("✗ Cannot create VideoWriter for test")
        
except Exception as e:
    print(f"✗ Test video creation error: {e}")

print("\n=== SUMMARY ===")
print("If you still don't see downloadable files:")
print("1. Check the 'Output' tab in your Kaggle notebook")
print("2. Look for files in both /kaggle/output/ and /kaggle/working/")
print("3. Try restarting the kernel and running just the file copying part")
print("4. The videos might be in a different location - check the full file system")

Using device: cuda
Self-Correcting Video Continuation System
Created /kaggle/output directory
Found video files: ['/kaggle/input/my-videos-for-continuation/211056_crow_swooping_down_to_pick_final.mp4', '/kaggle/input/my-videos-for-continuation/210417_crow_swooping_down_to_pick_shot9.mp4', '/kaggle/input/my-videos-for-continuation/213459_crow_carrying_gold_coin_shot1.mp4', '/kaggle/input/my-videos-for-continuation/201744_crow_swooping_down_to_pick_shot2.mp4', '/kaggle/input/my-videos-for-continuation/205738_crow_swooping_down_to_pick_shot8.mp4', '/kaggle/input/my-videos-for-continuation/214943_crow_carrying_gold_coin_shot3.mp4', '/kaggle/input/my-videos-for-continuation/215705_crow_carrying_gold_coin_shot4.mp4', '/kaggle/input/my-videos-for-continuation/214220_crow_carrying_gold_coin_shot2.mp4', '/kaggle/input/my-videos-for-continuation/211056_crow_swooping_down_to_pick_shot10.mp4', '/kaggle/input/my-videos-for-continuation/203103_crow_swooping_down_to_pick_shot4.mp4', '/kaggle/input/my

100%|██████████| 30/30 [00:01<00:00, 23.79it/s]


Frame 6: High error detected (0.496), applying correction
Frame 7: High error detected (0.496), applying correction
Frame 8: High error detected (0.496), applying correction
Frame 9: High error detected (0.496), applying correction
Frame 10: High error detected (0.496), applying correction
Frame 11: High error detected (0.496), applying correction
Frame 12: High error detected (0.496), applying correction
Frame 13: High error detected (0.496), applying correction
Frame 14: High error detected (0.496), applying correction
Frame 15: High error detected (0.496), applying correction
Frame 16: High error detected (0.496), applying correction
Frame 17: High error detected (0.496), applying correction
Frame 18: High error detected (0.496), applying correction
Frame 19: High error detected (0.496), applying correction
Frame 20: High error detected (0.496), applying correction
Frame 21: High error detected (0.496), applying correction
Frame 22: High error detected (0.496), applying correction
F

100%|██████████| 30/30 [00:00<00:00, 274.29it/s]

Frame 6: High error detected (0.496), applying correction
Frame 7: High error detected (0.496), applying correction
Frame 8: High error detected (0.496), applying correction
Frame 9: High error detected (0.496), applying correction
Frame 10: High error detected (0.496), applying correction
Frame 11: High error detected (0.496), applying correction
Frame 12: High error detected (0.496), applying correction
Frame 13: High error detected (0.496), applying correction
Frame 14: High error detected (0.496), applying correction
Frame 15: High error detected (0.496), applying correction
Frame 16: High error detected (0.496), applying correction
Frame 17: High error detected (0.496), applying correction
Frame 18: High error detected (0.496), applying correction
Frame 19: High error detected (0.496), applying correction
Frame 20: High error detected (0.496), applying correction
Frame 21: High error detected (0.496), applying correction
Frame 22: High error detected (0.496), applying correction
F